In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

# Collections

In [ ]:
%load_ext autoreload
%autoreload
col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

# Artists

In [ ]:
%load_ext autoreload
%autoreload

def parseArtists(arts, force=False):
    for modVal in range(18,19):
        arts.parseArtistModValFiles(modVal, force=force)
        arts.parseArtistModValExtraFiles(modVal)

from artist import artist
from artists import artists
arts = artists(disc)

#parseArtists(arts, True)
#arts.parseArtistFiles(force=False)
#if False:
#art.searchDiscogForArtist("$uicideboy$")
arts.buildMetadata(force=False, doAlbums=True)

# New Artists From Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums

disc = discogs()
albs = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

In [ ]:
from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    print('\n{0}/{1}'.format(ia,len(refstoget)))
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            arts.downloadArtistURL(url, savename, debug=True)
        except:
            continue
        sleep(2)
    #print(item)
#len(togetmap)

# Albums

In [ ]:
def parseAlbums(modVal, force=False):
    albs.parseAlbumModValFiles(modVal=modVal, force=force)
    
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)


def f_amp(modVals=100):
    nProcs = 3
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, range(100))
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
albs.parseAlbumModValFiles(modVal='NAN', force=False)
f_amp()
elapsed(start2, cmt2)
albs.buildMetadata(force=False)
elapsed(start, cmt)

# New Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=3, mediaTypes=None, rand=True)

In [ ]:
for k,v in db.items():
    print(k,v)
    break

In [ ]:
f1="/Volumes/Music/Discog/albums/6/102506/482739.p"
#f2="/Volumes/Music/Discog/albums/6/102506/7303693.p"
#albData1 = alb.getData(f1)
f2="/Volumes/Music/Discog/albums/11/567511/3386791.p"
albData2 = alb.getData(f2)

In [ ]:
albData1.album

In [ ]:
for k,v in albData2.get().items():
    try:
        print(k,v.get())
    except:
        print(k,v)

In [ ]:
for k,v in albData1.get().items():
    try:
        print(k,v.get())
    except:
        print(k,v)

In [ ]:
from album import album
from fileUtils import getsize
alb = album()
for ifile in files:
    print(getsize(ifile))
    #alb.getData(ifile)

In [ ]:
for modVal in range(58,59):
    arts.parseArtistModValFiles(modVal=modVal, force=False)

dbArt = disc.getArtistIDToNameData()
dbCol = disc.getCollectionIDToRefData()
artistDBDir = disc.getArtistsDBDir()

In [ ]:
from time import sleep
refs = [dbCol.get('716158')]
for ref in refs:
    if ref is None:
        continue
    print("REF --> {0}".format(ref))
    href = ref.replace("https://www.discogs.com/artist/", "")
    artistID = dutils.getArtistID(ref)
    modValue = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())

    if True:
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)

        try:
            del dbdata[artistID]
            print("Deleted {0}".format(artistID))
        except:
            print("Not there...")

        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)

    url = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, parse=True, force=True, debug=True)
    sleep(1)

In [ ]:
import time
from multiprocessing import Pool
 
def f(a_list):
    out = 0
    for n in a_list:
        out += n*n
        time.sleep(0.1)
 
    return out
 
def f_mp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    pool = Pool(processes=5)
 
    result = pool.map(f, chunks)
 
    return sum(result)

In [ ]:
import time
from multiprocessing import Pool
 
def f(a_list):
    out = 0
    for n in a_list:
        out += n*n
        time.sleep(0.1)
 
    return out
 
def f_amp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    pool = Pool(processes=5)
 
    result = pool.map_async(f, chunks)
 
    while not result.ready():
        print("Running...")
        time.sleep(0.5)
 
    return sum(result.get())

In [ ]:
L = [{0: True}, {1: True}, {2: True}, {3: True}, {4: True}]

In [ ]:
{k: v for d in L for k, v in d.items()}

In [ ]:
import time
from multiprocessing import Pool
 
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()
artistInfo = artist()
artistDBDir = disc.getArtistsDBDir()   

In [ ]:
len(files)

In [ ]:
from fileUtils import getBaseFilename
import time
from multiprocessing import Pool

def f(finput):
    a_list = finput["Data"]
    dbdata = finput["DB"]
    out = []
    for ifile in a_list:
        discID = getBaseFilename(ifile)
        if dbdata.get(discID) is None:
            out.append(ifile)
            
    return out

def fDB(modVal):
    saveIt = arts.parseArtistModValFiles(modVal, force=True)
    
    if False:
        from searchUtils import findExt
        files = findExt(setDir(artistDir, str(modVal)), ext="*.p")
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        dbdata = getFile(dbname, version=3)

        out = []
        saveIt = 0
        for ifile in files:
            discID = getBaseFilename(ifile)
            if dbdata.get(discID) is None:
                info   = artistInfo.getData(ifile)
                dbdata[discID] = info
                saveIt += 1

        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))
            print("Saving {0} new artist IDs to {1}".format(saveIt, savename))
            saveJoblib(data=dbdata, filename=savename, compress=True)
        
    return saveIt

 
def f_ampFiles(a_list, dbdata):
    nProcs = 1
    chunks = [a_list[i::nProcs] for i in range(nProcs)]

    for c in chunks:
        print(len(c))
    
    pool = Pool(processes=nProcs)
    
    
 
    finputs = [{"Data": chunk, "DB": dbdata} for chunk in chunks]
    result = pool.map_async(f, finputs)
 
    start, cmt = clock("Processing {0} files using {1} processes".format(len(files), nProcs))
    print("Runing .",end="")
    while not result.ready():
        print(".",end="")
        time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()



 
def f_amp(modVals):
    nProcs = 3
    pool = Pool(processes=nProcs)
    result = pool.map_async(fDB, range(3,modVals))
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        print(".",end="")
        time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
from timeUtils import clock, elapsed
retval = f_amp(100)
#flat_list = [item for sublist in retval for item in sublist]
#print(len(flat_list))

In [ ]:
dbname = setFile(artistDBDir, "{0}-DB.p".format(67))            
dbdata = getFile(dbname, version=3)

In [ ]:
modVals = 1
nProcs = 1
start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
print("Runing .",end="")
retval = fDB(modVals)
print("")
elapsed(start, cmt)


In [ ]:
retval

In [ ]:
vals="""1,708,466 US
1,708,466 UK
1,154,284 Germany
637,261 France
493,079 Japan
490,698 Italy
434,681 Canada
433,543 Europe
416,965 Netherlands
339,738 Spain
320,714 Unknown
259,587 Russia
259,579 Australia
189,431 Sweden
164,278 Belgium
149,548 Brazil
131,106 Greece
124,488 Poland
121,249 Jamaica
105,887 Finland
86,538 Switzerland
85,145 USSR
80,203 Mexico
79,972 Denmark
77,131 Portugal
75,979 Norway
75,533 UK & Europe
67,292 Austria
63,672 Argentina"""

In [ ]:
countries = [" ".join(x.split()[1:]) for x in vals.split("\n")]
saveFile(idata=countries, ifile="countries.p")

In [ ]:
countries